In [1]:
from __future__ import annotations

import random
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Set

import numpy as np
import pandas as pd

# === USER CONFIG ===

# # 1) Profiling output from Section 3.1
# profiling_csv_path = "C:\\Users\\joel\\Desktop\\CAiSE_2\\DGP_1\\data\\profileprimitives_fooddeliver.csv"   # 👈 change as needed

# # 2) Relationship scores from Section 3.2
# relationship_csv_path = "C:\\Users\\joel\\Desktop\\CAiSE_2\\DGP_1\\data\\relationship_scores.csv"          # 👈 change as needed

# # 3) Folder containing raw CSV tables
# data_folder = "C:\\Users\\joel\\Desktop\\CAiSE_2\\DGP_1\\data\\food_deliver"                                      # 👈 change as needed

# # 4) Output CSV for traces
# traces_output_csv = "C:\\Users\\joel\\Desktop\\CAiSE_2\\DGP_1\\data\\case_traces.csv"                      # 👈 change as needed


# 1) Profiling output from Section 3.1
profiling_csv_path = "C:\\Users\\joel\\Desktop\\CAiSE_2\\TPC_H\\ProfilePrimitives_TPCH.csv"   # 👈 change as needed

# 2) Relationship scores from Section 3.2
relationship_csv_path = "C:\\Users\\joel\\Desktop\\CAiSE_2\\TPC_H\\relationship_scores.csv"          # 👈 change as needed

# 3) Folder containing raw CSV tables
data_folder = "C:\\Users\\joel\\Desktop\\CAiSE_2\\TPC_H\\tpch_csv"                                      # 👈 change as needed

# 4) Output CSV for traces
traces_output_csv = "C:\\Users\\joel\\Desktop\\CAiSE_2\\TPC_H\\case_traces.csv"                      # 👈 change as needed


# 5) How many cases to sample
N_CASES_TO_SAMPLE = 100

# Optional: random seed for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)



In [2]:
profiles = pd.read_csv(profiling_csv_path)
relationships = pd.read_csv(relationship_csv_path)

# Make sure boolean flags are really booleans
for col in ["is_pk_candidate", "is_dt_candidate"]:
    if col in profiles.columns:
        profiles[col] = profiles[col].astype(bool)
    else:
        profiles[col] = False  # fallback if missing
# PK candidates per table, sorted by s_pk descending
if "s_pk" not in profiles.columns:
    raise ValueError("Profiling CSV is missing 's_pk' column.")

pk_cols_by_table: Dict[str, List[str]] = (
    profiles[profiles["is_pk_candidate"]]
    .sort_values(["table", "s_pk"], ascending=[True, False])
    .groupby("table")["column"]
    .apply(list)
    .to_dict()
)

# Datetime candidates per table, sorted by s_dt descending
if "is_dt_candidate" not in profiles.columns or "s_dt" not in profiles.columns:
    raise ValueError("Profiling CSV must have 'is_dt_candidate' and 's_dt'.")

dt_cols_by_table: Dict[str, List[str]] = (
    profiles[profiles["is_dt_candidate"]]
    .sort_values(["table", "s_dt"], ascending=[True, False])
    .groupby("table")["column"]
    .apply(list)
    .to_dict()
)

print("PK candidates per table:")
for t, cols in pk_cols_by_table.items():
    print(f"  {t}: {cols}")

print("\nDatetime candidates per table:")
for t, cols in dt_cols_by_table.items():
    print(f"  {t}: {cols}")


required_rel_cols = {"table_i", "column_i", "table_k", "column_k", "s"}
missing = required_rel_cols - set(relationships.columns)
if missing:
    raise ValueError(f"Relationship CSV is missing columns: {missing}")

# Build adjacency list: (table, column) -> list of (neighbor_table, neighbor_column, score)
from collections import defaultdict

adjacency: Dict[Tuple[str, str], List[Tuple[str, str, float]]] = defaultdict(list)

for _, row in relationships.iterrows():
    t_i, c_i = row["table_i"], row["column_i"]
    t_k, c_k = row["table_k"], row["column_k"]
    s_val = float(row["s"])

    key1 = (t_i, c_i)
    key2 = (t_k, c_k)

    adjacency[key1].append((t_k, c_k, s_val))
    adjacency[key2].append((t_i, c_i, s_val))

len(adjacency)


PK candidates per table:
  customer: ['c_custkey', 'c_name', 'c_phone']
  nation: ['n_nationkey', 'n_comment', 'n_name']
  orders: ['o_orderkey', 'o_totalprice', 'o_comment']
  part: ['p_partkey', 'p_name']
  partsupp: ['ps_comment']
  region: ['r_regionkey', 'r_name', 'r_comment']
  supplier: ['s_suppkey', 's_name', 's_phone']

Datetime candidates per table:
  customer: ['c_acctbal', 'c_custkey', 'c_nationkey']
  lineitem: ['l_receiptdate', 'l_orderkey', 'l_shipdate']
  nation: ['n_nationkey', 'n_regionkey']
  orders: ['o_orderkey', 'o_orderdate', 'o_totalprice']
  part: ['p_partkey', 'p_retailprice', 'p_size']
  partsupp: ['ps_partkey', 'ps_suppkey', 'ps_availqty']
  region: ['r_regionkey']
  supplier: ['s_acctbal', 's_suppkey', 's_nationkey']


32

In [3]:
# Lazy cache of dataframes
_table_cache: Dict[str, pd.DataFrame] = {}

def load_table(table_name: str) -> Optional[pd.DataFrame]:
    """
    Load table <data_folder>/<table_name>.csv.
    Returns None if file is missing or unreadable.
    """
    if table_name in _table_cache:
        return _table_cache[table_name]

    path = Path(data_folder) / f"{table_name}.csv"
    if not path.exists():
        print(f"⚠️  Missing table file: {path}")
        return None

    try:
        df = pd.read_csv(path)
        _table_cache[table_name] = df
        return df
    except Exception as e:
        print(f"⚠️  Failed to read {path}: {e}")
        return None


In [4]:
from dataclasses import dataclass

@dataclass
class Event:
    case_id: str
    table: str
    time: pd.Timestamp
    dt_column: str
    row_index: int


def get_dt_columns(table: str) -> List[str]:
    """
    Return ordered list of datetime columns for a table,
    falling back to [] if none were identified.
    """
    return dt_cols_by_table.get(table, [])


In [5]:
import random
from typing import Optional, Tuple, List, Set

def sample_root_table() -> Optional[str]:
    """
    Choose a random table that has:
      - at least one PK candidate
      - at least one datetime candidate
      - a readable CSV file
    """
    valid_tables = []
    for t in pk_cols_by_table.keys():
        if t not in dt_cols_by_table:
            continue
        if load_table(t) is None:
            continue
        valid_tables.append(t)

    if not valid_tables:
        print("❌ No tables with both PK and datetime candidates.")
        return None

    return random.choice(valid_tables)

def normalize_ts(value) -> pd.Timestamp:
    """
    Parse any datetime-like value into a timezone-naive pandas.Timestamp.
    - Parsed with utc=True
    - Converted to tz-naive (UTC)
    Returns NaT if parsing fails.
    """
    ts = pd.to_datetime(value, errors="coerce", utc=True)
    if pd.isna(ts):
        return ts
    return ts.tz_convert(None)  # drop timezone, keep as naive UTC

    

def sample_case_from_root(root_table: str) -> Optional[Tuple[str, List[Event]]]:
    """
    Sample ONE case, using PK-centric propagation with local PK promotion:

    1. Randomly select a PK candidate column from `root_table`.
    2. Randomly select a PK value from that column.
    3. Explore (table, pk_col, pk_val) triples via BFS:
       - Within each visited table row, use *all* PK candidates in that row
         to seed new triples (step 5 of your logic).
       - Use relationship_scores adjacency to move between tables using
         joinable columns.

    Returns:
      (case_id_label, events_sorted) or None if no events found.
    """
    df_root = load_table(root_table)
    if df_root is None:
        return None

    pk_cols = pk_cols_by_table.get(root_table, [])
    if not pk_cols:
        return None

    # 1) Select a random PK column in the root table
    root_pk_col = random.choice(pk_cols)

    # 2) Select a random PK value from that column
    series = df_root[root_pk_col].dropna()
    if series.empty:
        return None

    root_value = random.choice(series.unique())
    root_val_str = str(root_value)
    case_id_label = f"{root_table}.{root_pk_col}={root_value}"

    # BFS queue over (table, pk_column, pk_value_as_str)
    queue: List[Tuple[str, str, str]] = []
    visited_pk_triplets: Set[Tuple[str, str, str]] = set()
    events: List[Event] = []

    queue.append((root_table, root_pk_col, root_val_str))
    visited_pk_triplets.add((root_table, root_pk_col, root_val_str))

    while queue:
        table, pk_col, pk_val_str = queue.pop(0)

        df = load_table(table)
        if df is None or pk_col not in df.columns:
            continue

        # 4) Select all rows where this PK value exists
        mask = df[pk_col].astype(str) == pk_val_str
        matches = df[mask]
        if matches.empty:
            continue

        dt_cols = dt_cols_by_table.get(table, [])
        table_pk_cols = pk_cols_by_table.get(table, [])

        for row_idx, row in matches.iterrows():
            # --- Emit events from this row (temporalization) ---
            for dt_col in dt_cols:
                if dt_col not in df.columns:
                    continue
                ts = normalize_ts(row[dt_col])
                if pd.isna(ts):
                    continue
                events.append(
                    Event(
                        case_id=case_id_label,
                        table=table,
                        time=ts,
                        dt_column=dt_col,
                        row_index=row_idx,
                    )
                )

            # --- 5) Local PK promotion: pick up *all* PKs in this row ---
            # e.g. from Deliveries row: DLV_ID, OrderRef, maybe others
            for local_pk_col in table_pk_cols:
                if local_pk_col not in df.columns:
                    continue
                new_val = row[local_pk_col]
                if pd.isna(new_val):
                    continue
                new_val_str = str(new_val)
                triplet_local = (table, local_pk_col, new_val_str)
                if triplet_local not in visited_pk_triplets:
                    visited_pk_triplets.add(triplet_local)
                    queue.append(triplet_local)

            # --- 3 & 6) Use relationships to hop to connected tables ---
            # Look at all joinable columns from this PK column
            for (nbr_table, nbr_col, score) in adjacency.get((table, pk_col), []):
                nbr_df = load_table(nbr_table)
                if nbr_df is None or nbr_col not in nbr_df.columns:
                    continue

                # 4) Check if this PK value appears in the neighbor join column
                nbr_mask = nbr_df[nbr_col].astype(str) == pk_val_str
                nbr_matches = nbr_df[nbr_mask]
                if nbr_matches.empty:
                    continue

                # 5) From those neighbor rows, pick up their PK candidates
                nbr_pk_cols = pk_cols_by_table.get(nbr_table, [])
                if not nbr_pk_cols:
                    continue

                for n_idx, n_row in nbr_matches.iterrows():
                    # Emit neighbor events
                    nbr_dt_cols = dt_cols_by_table.get(nbr_table, [])
                    for dt_col in nbr_dt_cols:
                        if dt_col not in nbr_df.columns:
                            continue
                        ts = normalize_ts(n_row[dt_col])
                        if pd.isna(ts):
                            continue
                        events.append(
                            Event(
                                case_id=case_id_label,
                                table=nbr_table,
                                time=ts,
                                dt_column=dt_col,
                                row_index=n_idx,
                            )
                        )

                    # Promote neighbor PKs
                    for nbr_pk_col in nbr_pk_cols:
                        if nbr_pk_col not in nbr_df.columns:
                            continue
                        new_val = n_row[nbr_pk_col]
                        if pd.isna(new_val):
                            continue
                        new_val_str = str(new_val)
                        triplet = (nbr_table, nbr_pk_col, new_val_str)
                        if triplet not in visited_pk_triplets:
                            visited_pk_triplets.add(triplet)
                            queue.append(triplet)

    if not events:
        return None

    # Deduplicate repeated captures of the *same* business event
    unique_events = {}
    for e in events:
        key = (e.table, e.dt_column, e.time)
        if key not in unique_events:
            unique_events[key] = e

    events_dedup = list(unique_events.values())

    # Stable time sort → ε_s
    events_sorted = sorted(events_dedup, key=lambda e: (e.time, e.table, e.row_index))
    return case_id_label, events_sorted





In [6]:
def build_case_traces(
    n_cases: int,
    allow_duplicates: bool = False,
) -> pd.DataFrame:
    """
    Build `n_cases` sampled traces.

    Output columns:
        case_id, root_table, root_pk_column, event_sequence, num_events
    """
    records = []
    seen_case_ids: Set[str] = set()

    while len(records) < n_cases:
        root_table = sample_root_table()
        if root_table is None:
            break

        result = sample_case_from_root(root_table)
        if result is None:
            continue

        case_id_label, events_sorted = result
        if (not allow_duplicates) and (case_id_label in seen_case_ids):
            continue
        seen_case_ids.add(case_id_label)

        # Full table sequence (recurrences allowed, but deduped per timestamp)
        table_sequence = [e.table for e in events_sorted]
        sequence_str = " > ".join(table_sequence)

        # Extract root table and root pk column from label
        try:
            root_info, value_str = case_id_label.split("=", 1)
            r_table, r_col = root_info.split(".", 1)
        except ValueError:
            r_table = root_table
            r_col = pk_cols_by_table.get(root_table, ["<unknown>"])[0]

        records.append(
            dict(
                case_id=case_id_label,
                root_table=r_table,
                root_pk_column=r_col,
                event_sequence=sequence_str,
                num_events=len(events_sorted),
            )
        )

    return pd.DataFrame.from_records(records)


In [ ]:
case_traces_df = build_case_traces(N_CASES_TO_SAMPLE)
print(f"Built {len(case_traces_df)} case traces.")
case_traces_df.head()

case_traces_df.to_csv(traces_output_csv, index=False)
print(f"Case traces written to: {traces_output_csv}")
